In [2]:


import numpy as np
import gym
from gym import spaces
from gym.spaces import Box, Dict, Discrete, MultiBinary, MultiDiscrete
import numpy as np
from stable_baselines3 import PPO,A2C
import numpy as np
import pandas as pd
from IPython.display import display 
import nltk
from nltk import sent_tokenize,word_tokenize
import itertools
from nltk.tokenize.treebank import TreebankWordDetokenizer

# the cut -in windspeed (km/h =1/3.6 m/s), v^ci#
cutin_windspeed = 3*3.6
# the cut -off windspeed (km/h =1/3.6 m/s), v^co#
cutoff_windspeed = 11*3.6
# the rated windspeed (km/h =1/3.6 m/s), v^r#
rated_windspeed = 7*3.6
# the charging - discharging efficiency , eta#
charging_discharging_efficiency = 0.95
# the rate for discharging the battery ( MegaWatt ), b#
rate_battery_discharge = 2/1000
# the unit operational and maintanance cost for generating power
# from solar PV (10^4 $/ MegaWattHour =10 $/ kWHour ), r_omc ^s#
unit_operational_cost_solar = 0.15/10
# the unit operational and maintanance cost for generating power
# from wind turbine (10^4 $/ MegaWattHour =10 $/ kWHour ), r_omc ^w#
unit_operational_cost_wind = 0.085/10
# the unit opeartional and maintanance cost for generating power
# from generator (10^4 $/ MegaWattHour =10 $/ kWHour ), r_omc ^g#
unit_operational_cost_generator = 0.55/10
# the unit operational and maintanance cost for battery storage system
# per unit charging / discharging cycle (10^4 $/ MegaWattHour =10 $/ kWHour ),
# r_omc ^b#
unit_operational_cost_battery = 0.95/10
# the capacity of battery storage system ( MegaWatt Hour =1000 kWHour ), e#
capacity_battery_storage = 300/1000
# the maximum state of charge of battery system #
SOC_max = 0.95* capacity_battery_storage
# the minimum state of charge of battery system #
SOC_min = 0.05* capacity_battery_storage
# the area of the solar PV system (km ^2=1000*1000 m^2) , a#
area_solarPV = 1400/(1000*1000)
# the efficiency of the solar PV system , delta #
efficiency_solarPV = 0.2
# calculate the rated power of the wind turbine ,
# density of air (10^6 kg/km ^3=1 kg/m^3) , rho#
density_of_air = 1.225
# calculate the rated power of the wind turbine ,
# radius of the wind turbine blade (km =1000 m), r#
radius_wind_turbine_blade = 25/1000
# calculate the rated power of the wind turbine ,
# average wind speed (km/h =1/3.6 m/s), v_avg ( from the windspeed table )#
average_wind_speed = 3.952*3.6
# calculate the rated power of the wind turbine , power coefficient , theta #
power_coefficient = 0.593
# calculate the rated power of the wind turbine ,
# gearbox transmission efficiency , eta_t #
gearbox_transmission_efficiency = 0.95
# calculate the rated power of the wind turbine ,
# electrical generator efficiency , eta_g #
electrical_generator_efficiency = 0.95
# the rated power of the wind turbine , RP_w ( MegaWatt =10^6 W),
# with the radius_wind_turbine_blade measured in km =10^3m,
# average wind speed measured in km/ hour =3.6 m/s,
# RP_w will be calculated as RP_w_numerical
# then RP_w in MegaWatt =(1 kg/m^3) *(10^3 m) *(10^3 m)
# *(3.6 m/s) *(3.6 m/s) *(3.6 m/s)* RP_w_numerical
# =3.6^3*10^6 RP_w_numerical W =3.6^3 RP_w_numerical MegaWatt #
rated_power_wind_turbine_original =\
    (0.5 * density_of_air * np.pi
     * radius_wind_turbine_blade
     * radius_wind_turbine_blade
     * average_wind_speed
     * average_wind_speed
     * average_wind_speed
     * power_coefficient
     * gearbox_transmission_efficiency
     * electrical_generator_efficiency)
rated_power_wind_turbine = rated_power_wind_turbine_original / (3.6*3.6*3.6)
# the number of wind turbine in the onsite generation system , N_w#
number_windturbine = 1
# the number of generators , n_g#
rate_battery_discharge = 1
number_generators=1
# the rated output power of the generator ( MegaWatt =1000 kW), G_p#
rated_output_power_generator = 60/1000

unit_reward_soldbackenergy=0.2





# read the solar irradiance and wind speed data from file #
# read the rate of consumption charge date from file #
file_SolarIrradiance = "SolarIrradiance.csv"
file_WindSpeed = "WindSpeed.csv"
file_rateConsumptionCharge = "rate_consumption_charge.csv"
# read the solar irradiace
data_solar = pd.read_csv(file_SolarIrradiance)

# solar irradiance measured by MegaWatt /km ^2
solarirradiance = np.array( data_solar.iloc[: ,3].dropna())
# read the windspeed
data_wind = pd.read_csv( file_WindSpeed )
windspeed = 3.6* np.array( data_wind.iloc[: ,3].dropna())# windspeed measured by km/h =1/3.6 m/s

# read the rate of consumption charge
data_rate_consumption_charge =pd.read_csv( file_rateConsumptionCharge ).dropna()
#rate_consumption_charge = np.array( data_rate_consumption_charge.iloc[: ,4])/10
# rate of consumption charge measured by 10^4 $/ MegaWatt =10 $/kWh
house_hold_data=pd.read_csv("RefBldgFullServiceRestaurantNew2004_v1.3_7.1_8A_USA_AK_FAIRBANKS.csv")


P_g_t=np.array( data_rate_consumption_charge.iloc[: ,4])/10 #cost of using windmills,and PV and genrerator for microgrid

tokenize_col=house_hold_data.columns



tokenized_sentences_list = [sent_tokenize(text) for text in tokenize_col]
tokenized_sentences_list = [word_tokenize(text) for text in tokenize_col]


occurrences_electricity = np.array([(i, j) for i, sublist in enumerate(tokenized_sentences_list) for j, word in enumerate(sublist) if word == 'Electricity'])[:,0]
occurrences_gas = np.array([(i, j) for i, sublist in enumerate(tokenized_sentences_list) for j, word in enumerate(sublist) if word == 'Gas'])[:,0]


Electricity_consumption_wind_solar=house_hold_data[tokenize_col[occurrences_electricity]].sum(axis=1)
Electricity_gas=house_hold_data[tokenize_col[occurrences_gas]].sum(axis=1)
Electricity_gas
total_energy=Electricity_gas+Electricity_consumption_wind_solar



def generate_binary_combinations(num_vars):
    # Generate all binary combinations for num_vars as a list of lists
    binary_combinations = np.unpackbits(np.arange(2**num_vars, dtype=np.uint8)[:, np.newaxis], axis=1)
    binary_combinations = binary_combinations[:, -num_vars:]
    binary_combinations_list = binary_combinations.tolist()
    return binary_combinations_list




class Microgrid:
    def __init__(self,
                 workingstatus=np.array([0, 0, 0]),
                 SOC=0,
                 actions_adjustingstatus=np.array([0, 0, 0]),
                 actions_solar=np.array([0, 0, 0]),
                 actions_wind=np.array([0, 0, 0]),
                 actions_generator=np.array([0, 0, 0]),
                 actions_purchased=np.array([0, 0]),
                 actions_discharged=0,
                 solarirradiance=solarirradiance,
                 windspeed=windspeed):
        self.workingstatus = workingstatus
        self.SOC = SOC
        self.actions_adjustingstatus = actions_adjustingstatus
        self.actions_solar = actions_solar
        self.actions_wind = actions_wind
        self.actions_generator = actions_generator
        self.actions_purchased = actions_purchased
        self.actions_discharged = actions_discharged
        self.solarirradiance = solarirradiance
        self.windspeed = windspeed
        self.count=-1
    def transition(self):
       
        workingstatus = self.workingstatus
        SOC = self.SOC
        if self.actions_adjustingstatus[0] == 1:
            workingstatus[0] = 1
        else:
            workingstatus[0] = 0
        
        if self.actions_adjustingstatus[1] == 1 or (self.solarirradiance[self.count] * efficiency_solarPV * efficiency_solarPV / 1000 > 0):
            workingstatus[1] = 1
        else:
            if self.actions_adjustingstatus[1] == 1 and cutoff_windspeed >= self.windspeed[self.count] >= cutin_windspeed:
                workingstatus[1] = 1
        
        if self.actions_adjustingstatus[2] == 1:
            workingstatus[2] = 1
        else:
            workingstatus[2] = 0
        
        SOC = (self.SOC + (self.actions_solar[1] + self.actions_wind[1] + self.actions_generator[1]
                           + self.actions_purchased[1]) * charging_discharging_efficiency
               - self.actions_discharged / charging_discharging_efficiency)
        #print(SOC)
        SOC = max(min(SOC, SOC_max), SOC_min)
        
        return workingstatus, SOC

    def EnergyConsumption(self):

     
        return -(self.actions_solar[0] + self.actions_wind[0] + self.actions_generator[0] + self.actions_discharged)

    def energy_generated_solar(self):
        #print(self.solarirradiance[self.count])
        if self.workingstatus[0] == 1:
            energy_generated_solar = self.solarirradiance[self.count] * area_solarPV * efficiency_solarPV / 1000
        else:
            energy_generated_solar = 0
        return energy_generated_solar

    def energy_generated_wind(self):
        Delta_t=1
        if self.workingstatus[1] == 1 and rated_windspeed > self.windspeed[self.count] >= cutin_windspeed:
            energy_generated_wind = (number_windturbine
                                     * rated_power_wind_turbine
                                     * (self.windspeed[self.count] - cutin_windspeed) / (rated_windspeed - cutin_windspeed))
        else:
            if self.workingstatus[1] == 1 and cutoff_windspeed > self.windspeed[self.count] >= rated_windspeed:
                energy_generated_wind = number_windturbine * rated_power_wind_turbine * Delta_t
            else:
                energy_generated_wind = 0
        return energy_generated_wind

    def energy_generated_generator(self):
        Delta_t=1
        if self.workingstatus[2] == 1:
            energy_generated_generator = number_generators * rated_output_power_generator * Delta_t
        else:
            energy_generated_generator = 0
        return energy_generated_generator

    def OperationalCost(self):
        Delta_t=1
        energy_generated_solar = self.energy_generated_solar()
        energy_generated_wind = self.energy_generated_wind()
        energy_generated_generator = self.energy_generated_generator()
        
        operational_cost = (energy_generated_solar * unit_operational_cost_solar
                            + energy_generated_wind * unit_operational_cost_wind
                            + energy_generated_generator * unit_operational_cost_generator)
        
        operational_cost += ((self.actions_discharged + self.actions_solar[1] + self.actions_wind[1] +
                              self.actions_generator[1]) * Delta_t * unit_operational_cost_battery /
                             (2 * capacity_battery_storage * (SOC_max - SOC_min)))
        
        return operational_cost

    def SoldBackReward(self):
      
        return ((self.actions_solar[2] + self.actions_wind[2] + self.actions_generator[2]) *
                unit_reward_soldbackenergy)

    def PrintMicrogrid(self, file):
        print("Microgrid working status [solar PV, wind turbine, generator] =", self.workingstatus, ", SOC =", self.SOC, file=file)
        print("Microgrid actions [solar PV, wind turbine, generator] =", self.actions_adjustingstatus, file=file)
        print("Solar energy supporting [the energy load, charging battery, sold back] =", self.actions_solar, file=file)
        print("Wind energy supporting [the energy load, charging battery, sold back] =", self.actions_wind, file=file)
        print("Generator energy supporting [the energy load, charging battery, sold back] =", self.actions_generator, file=file)
        print("Energy purchased from grid supporting [the energy load, charging battery] =", self.actions_purchased, file=file)
        print("Energy discharged by the battery supporting the energy load =", self.actions_discharged, file=file)
        print("Solar irradiance =", self.solarirradiance, file=file)
        print("Wind speed =", self.windspeed, file=file)
        print("Microgrid Energy Consumption =", self.EnergyConsumption(), file=file)
        print("Microgrid Operational Cost =", self.OperationalCost(), file=file)
        print("Microgrid SoldBackReward =", self.SoldBackReward(), file=file)
        print("\n", file=file)
        return None
        
        




class CustomEnv(gym.Env):
    def __init__(self, price, binary_combinations_1, binary_combinations_2, binary_combinations_3,P_g_t,windspeed,solarirradiance,total_energy,number_generators):
        super(CustomEnv, self).__init__()
        self.price = price
        self.binary_combinations_1 = binary_combinations_1
        self.binary_combinations_2 = binary_combinations_2
        self.binary_combinations_3 = binary_combinations_3
        self.microgrid_instance = None

        self.observation_space = spaces.Box(low=np.array([0, 0, 0, 0]), high=np.array([1, 1, 1, 1000]), dtype=np.float32)
        self.action_space = MultiDiscrete([8, 8, 8, 8, 4, 1])

    def step(self, action):
       
        
        self.microgrid_instance.count+=1
        adjusting_status = self.binary_combinations_3[action[0]]
        solar_actions = self.binary_combinations_3[action[1]]
        wind_actions = self.binary_combinations_3[action[2]]
        generator_actions = self.binary_combinations_3[action[3]]
        purchased_actions = self.binary_combinations_2[action[4]]
        purchased_discharged = self.binary_combinations_1[action[5]]

        """
        adjusting_status = action[0:3]
        solar_actions = action[3:6]
        wind_actions = action[6:9]
        generator_actions = action[9:12]
        purchased_actions = action[12:14]
        purchased_discharged = action[14]
        """
        self.microgrid_instance.actions_adjustingstatus = np.array(adjusting_status)
        self.microgrid_instance.actions_solar = np.array(solar_actions)
        self.microgrid_instance.actions_wind = np.array(wind_actions)
        self.microgrid_instance.actions_generator = np.array(generator_actions)
        self.microgrid_instance.actions_purchased = np.array(purchased_actions)
        self.microgrid_instance.actions_discharged = purchased_discharged[0]*rate_battery_discharge  # Assuming action[7] corresponds to actions_discharged
        """
        print("Actions Solar Shape:", self.microgrid_instance.actions_solar.shape)
        print("Actions Wind Shape:", self.microgrid_instance.actions_wind.shape)
        print("Actions Generator Shape:", self.microgrid_instance.actions_generator.shape)
        print("Actions Purchased Shape:", self.microgrid_instance.actions_purchased.shape)
        """
        #print(self.microgrid_instance.transition())
        self.state_working_space,self.state_Soc = self.microgrid_instance.transition()
        self.state=np.concatenate((self.state_working_space, [self.state_Soc])).astype(int)
       
        self.S_t = self.microgrid_instance.SoldBackReward()
        self.O_t = self.microgrid_instance.OperationalCost()
        self.E_t =  self.microgrid_instance.energy_generated_solar()+self.microgrid_instance.energy_generated_wind()+self.microgrid_instance.energy_generated_generator()
        Rest_after_consumption = (self.E_t - total_energy[self.microgrid_instance.count])*self.microgrid_instance.EnergyConsumption()
        
        self.reward = self.E_t*P_g_t[self.microgrid_instance.count] + self.O_t - self.S_t
        
        
        #print(P_g_t[self.microgrid_instance.count])
        
        done=False
        if self.microgrid_instance.count==10:
           done = True
        #print( self.reward )
        return self.state, self.reward, done, {}
        
    def reset(self):
        self.microgrid_instance = Microgrid()  # Create a new Microgrid instance for each episode
        self.state_working_space,self.state_Soc = self.microgrid_instance.transition()
        self.state=np.concatenate((self.state_working_space, [self.state_Soc])).astype(int)
        #print(self.state)
        return self.state

    def render(self):
        pass

price = 2 # Replace with the appropriate price value

num_binary_vars = 3  # Change this to the number of binary variables you have
binary_combinations_3 = generate_binary_combinations(num_binary_vars)

num_binary_vars = 2  # Change this to the number of binary variables you have
binary_combinations_2 = generate_binary_combinations(num_binary_vars)

num_binary_vars = 1  # Change this to the number of binary variables you have
binary_combinations_1 = generate_binary_combinations(num_binary_vars)


env = CustomEnv(price, binary_combinations_1, binary_combinations_2, binary_combinations_3,np.array(P_g_t), np.array(windspeed),np.array(solarirradiance),np.array(total_energy),number_generators)


model = A2C("MlpPolicy", env, verbose=1)

model.learn(total_timesteps=int(1e2))

learned_policy = model.policy

obs = env.reset()  # Reset the environment to get the initial observation/state
done = False  # Set the done flag to False initially

while not done:
    action, _ = learned_policy.predict(obs, deterministic=True)  # Predict action based on the current observation
    obs, reward, done, _ = env.step(action)

print(obs,reward,done) 




C:\Users\rasyd\anaconda3\envs\micro\lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


Using cpu device


C:\Users\rasyd\anaconda3\envs\micro\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
[1 1 1 0] 1.5631374812763383 True
